# 梯度裁剪防止梯度消失或者爆炸
# 也可以堪称一个分类问题，预测的输出层是词汇表中每一个词出现的概率

In [1]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)  # Return the iterator and the vocabulary of the time machine dataset.

In [2]:
vocab

In [3]:
len(vocab)

28

In [4]:
# 生成len(vocab)长度的独热码，显示tensor存放下标第几个
F.one_hot(torch.tensor([0,1,2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [6]:
X = torch.arange(10).reshape((2, 5))  # 批量大小，时间
F.one_hot(X.T, 28).shape  # 时间，批量，特征长度
# 这里的形状理解成，2个句子，每个句子包含5个单词，每个单词的特征维度用28表示

torch.Size([5, 2, 28])

In [7]:
# vocab_size词汇表长度，对应one-hot的维度，同时也是输入词汇转成独热码的维度，这几个长度统一
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01  # 均值0，方差0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))  # input->hidden
    W_hh = normal((num_hiddens, num_hiddens))  # hidden->hidden
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))  # hidden2->output
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)  # 给每个参数增加梯度
    return params

In [8]:
# 初始化隐藏层参数
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [9]:
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:  # 对于每一个时间步
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h) # 先更新隐藏状态，根据当前输入和上一时间隐层
        Y = torch.mm(H, W_hq) + b_q
        print(H.shape)
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

+ Wxh-n*hidden
+ Whh-hidden*hidden
+ b_h-1*hidden
+ Whq-hidden*n
+ b_q-1*n
+ inputs-t*b*n，在一个时间上维度b*n
前三个参数更新隐藏层（更新时间信息），后两个则输出
单个时间步长，更新隐藏层，time*b*n-n*hidden+hidden*hidden-hidden*hidden+1*hidden=hidden*hidden
输出 hidden*hidden-hidden*n+1*n=hidden*n-》batch*hidden*n

In [10]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [18]:
num_hiddens = 512 # 隐藏层参数是一个超参数
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)  # 调用__call__，结果是输出和隐藏层
new_state

torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 512])


(tensor([[-0.0034,  0.0086, -0.0009,  ...,  0.0121,  0.0145, -0.0208],
         [ 0.0099, -0.0149, -0.0099,  ..., -0.0131, -0.0167, -0.0043]],
        device='cuda:0', grad_fn=<TanhBackward0>),)

In [14]:
X.shape[0]

2

In [18]:
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save  # prefix-一段开头，num_preds要预测的数量，net-训练好的神经网络
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

# 简洁实现

In [19]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [20]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

In [21]:

state = torch.zeros((1, batch_size, num_hiddens))
state.shape

torch.Size([1, 32, 256])

In [22]:
X = torch.rand(size=(num_steps, batch_size, len(vocab)))
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))